In [9]:
import datetime
import os
import sys
import pandas as pd
import numpy as np
import xgboost
from sklearn.model_selection import train_test_split
import dataframe_image as dfi
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


notebook_dir = os.path.abspath(os.getcwd())
src_path = os.path.join('/home/lerceg/LukaPosao/market_models/', 'src')
sys.path.append(src_path)

import market_models.src.models.train_model as tm
import market_models.src.models.predict_model as pm
import market_models.src.utils as utils

%load_ext autoreload
%autoreload 
import market_models.src.features.build_features as build_features


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
other_path = utils.get_others_path("netrisk_casco")
eurotax = pd.read_csv(f"{other_path}full_trim_list.csv", low_memory=False)
prices = pd.read_csv(f"{other_path}price.csv", low_memory=False)[['eurotax_code', 'new_price_1_gross']]

In [11]:

eurotax = pd.merge(eurotax, prices, on='eurotax_code')
eurotax['new_price_1_gross'] *= utils.FORINT_TO_EUR
eurotax = eurotax[eurotax['vehicle_type'] == 10]
netrisk_cars = pd.read_csv(f"{other_path}netrisk_cars.csv", low_memory=False)

In [12]:
netrisk_cars = netrisk_cars.drop_duplicates('eurotax_code')
netrisk_cars = netrisk_cars[(netrisk_cars['car_trim_id'] == 1) & (netrisk_cars['car_make_year'] >= 2014)]

In [13]:
postal_codes = pd.read_csv(f'{utils.get_others_path("netrisk_casco")}hungary_postal_codes.csv')['postal_code']
population = pd.read_csv(f'{utils.get_others_path("netrisk_casco")}hungary_postal_codes_population.csv').rename(
    columns={'PostalCode': 'postal_code',
             'Population': 'population'})[['postal_code', 'population']]


def to_int(x):
    try:
        return int(str(x).replace(',', ''))
    except Exception as e:
        return None


population['population'] = population['population'].apply(to_int)
population = population.dropna()


In [ ]:

base_profile = utils.read_file(utils.get_profiles_for_crawling_transposed("netrisk_casco_sampled_data"))
base_profile[(base_profile['CarModel'] == 'Corolla')]
base_profile = base_profile.iloc[78]
base_profile

In [ ]:

base_profile['Age'] = 35
base_profile['CarAge'] = 1
base_profile['BonusMalus'] = 'B10'
base_profile['BonusMalusCode'] = 1
base_profile['PostalCode'] = 1011
base_profile['CarModelSpecific'] = 1
base_profile.to_csv(utils.get_incremental_base_profile_path("netrisk_casco", "v1"))

In [18]:

population = population[population['population'].between(2000, 1000000)]

postal_codes = population['postal_code'].values.tolist()
values = pd.DataFrame(columns=['feature', 'min', 'max', 'step', 'values'])
#features_to_vary = pd.Series(['PostalCode', 'Age', 'CarAge', 'BonusMalus', ('CarAge', 'CarMake', 'CarModel', 'CarModelSpecific')])
features_to_vary = pd.Series(['PostalCode', 'Age', 'CarAge', 'BonusMalus'])
#features_to_vary = pd.Series(['PostalCode'])
car_values = netrisk_cars[['car_make_year', 'car_make', 'car_model', 'car_trim_id']].values
car_values[:, 0] = datetime.datetime.today().year - car_values[:, 0]
car_values = list(map(tuple, car_values))
#feature_values = pd.Series([postal_codes, None, None, utils.BONUS_MALUS_CLASSES_GOOD, car_values])
feature_values = pd.Series([postal_codes, None, None, utils.BONUS_MALUS_CLASSES_GOOD])
#feature_values = pd.Series([postal_codes])
values['values'] = feature_values
values['feature'] = features_to_vary
values['min'] = pd.Series([None, 19, 0, None])
values['max'] = pd.Series([None, 79, 10, None])
values['step'] = pd.Series([None, 2, 1, None])
values.to_csv(utils.get_incremental_values_path("netrisk_casco", "v5"))

In [19]:
len(postal_codes)

966